<a href="https://colab.research.google.com/github/sidharthmrao/Chess_AI_Bot/blob/master/ChessAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import copy

##CHESS FRAME

class frameboard():


  #Creates a gameboard in its original state
  def createboard():
    #creates a dictionary of all parts and some booleans:
    # The value is the piece name
    # The first item in the list is the same
    # The second is whether the piece is captured or active (dead or alive)
    # The third is only for kings and rooks; it tells you if they moved or not, helping in castling
    pieces = {'Rb1'   :   ['Rb1'  , 'Active', False], 
              'KNb1'  :   ['KNb1' , 'Active' ],
              'Bb1'   :   ['Bb1'  , 'Active'], 
              'Qb'    :   ['Qb'   , 'Active'], 
              'Ib'    :   ['Ib'   , 'Active', False], 
              'Bb2'   :   ['Bb2'  , 'Active'], 
              'KNb2'  :   ['Rb1'  , 'Active'], 
              'Rb2'   :   ['Rb2'  , 'Active', False],
              'Pb1'   :   ['Pb1'  , 'Active'],
              'Pb2'   :   ['Pb2'  , 'Active'],
              'Pb3'   :   ['Pb3'  , 'Active'],
              'Pb4'   :   ['Pb4'  , 'Active'],
              'Pb5'   :   ['Pb5'  , 'Active'],
              'Pb6'   :   ['Pb6'  , 'Active'],
              'Pb7'   :   ['Pb7'  , 'Active'],
              'Pb8'   :   ['Pb8'  , 'Active'],

              'Rw1'   :   ['Rw1'  , 'Active', False],
              'KNw1'  :   ['KNw1' , 'Active'],
              'Bw1'   :   ['Bw1'  , 'Active'], 
              'Qw'    :   ['Qw'   , 'Active'], 
              'Iw'    :   ['Iw'   , 'Active', False], 
              'Bw2'   :   ['Bw2'  , 'Active'], 
              'KNw2'  :   ['Rw1'  , 'Active'], 
              'Rw2'   :   ['Rw2'  , 'Active', False],
              'Pw1'   :   ['Pw1'  , 'Active'],
              'Pw2'   :   ['Pw2'  , 'Active'],
              'Pw3'   :   ['Pw3'  , 'Active'],
              'Pw4'   :   ['Pw4'  , 'Active'],
              'Pw5'   :   ['Pw5'  , 'Active'],
              'Pw6'   :   ['Pw6'  , 'Active'],
              'Pw7'   :   ['Pw7'  , 'Active'],
              'Pw8'   :   ['Pw8'  , 'Active'],

              'blk'   :   ['  '   , 'Active'] }                                            
    # This is the board. Blk means blank. The first letter (or two letters in the case of Knights) signifies the piece
    #The next letter signifies the team
    board = [['Rb1', 'KNb1', 'Bb1', 'Qb' , 'Ib' , 'Bb2', 'KNb2', 'Rb2' ],
             ['Pb1', 'Pb2' , 'Pb3', 'Pb4', 'Pb5', 'Pb6', 'Pb7' , 'Pb8' ],
             ['blk', 'blk' , 'blk', 'blk', 'blk', 'blk', 'blk' , 'blk' ],
             ['blk', 'blk' , 'blk', 'blk', 'blk', 'blk', 'blk' , 'blk' ],
             ['blk', 'blk' , 'blk', 'blk', 'blk', 'blk', 'blk' , 'blk' ],
             ['blk', 'blk' , 'blk', 'blk', 'blk', 'blk', 'blk' , 'blk' ],
             ['Pw1', 'Pw2' , 'Pw3', 'Pw4', 'Pw5', 'Pw6', 'Pw7' , 'Pw8' ],
             ['Rw1', 'KNw1', 'Bw1', 'Qw' , 'Iw' , 'Bw2', 'KNw2', 'Rw2' ]]

    board = np.asarray(board) #Formats into numpy array for easy display
    return pieces, board

class moves():


  #Verifies the value of a location (can it be moved into, is there another piece there, if so, from team of piece or opposing team?)
  def verify(move, piece, board):

    if move[0]<0 or move[0]>7 or move[1]<0 or move[1]>7:
      return 'impossible'

    spot = board[move[0], move[1]]
    piece = board[piece[0], piece[1]]

    if spot == 'blk':
      return 'blank'

    elif (('w' in piece) and ('b' in spot)) or (('b' in piece) and ('w' in spot)):
      return spot

    else:
      return 'impossible'

    # Remove all Impossible Moves
  def removeimp(possiblemoves): 
    temp = []
    for move in possiblemoves:
      if move[2] != "impossible":
        temp.append(move)
    return temp
   # All moves of a pawn 
  def pawn(piece, board):
    row = piece[0]
    column = piece[1]
    possiblemoves = []
    if 'b' in board[row][column]: #if piece is black
      if row == 1: #and in row 1
        possiblemoves = [[row+1, column], [row+2, column], [row+1, column+1], [row+1, column-1]]  #it can move up two, up 1, or diagonal by 1
      else: #but not in row 1
        possiblemoves = [[row+1, column], [row+1, column+1], [row+1, column-1]] #it can move up 1, or diagonal by 1


    elif 'w' in board[row][column]:# same for white
      if row == 6:
        possiblemoves = [[row-1, column], [row-2, column], [row-1, column+1], [row-1, column-1]]
      else:
        possiblemoves = [[row-1, column], [row-1, column+1], [row-1, column-1]]

    #pass through verify function in order to check to make sure the move is valid
    for possiblemove in possiblemoves:
      possiblemove.append(moves.verify(possiblemove, piece, board))

    if len(possiblemoves)==4: #if on first row
      #The last 2 moves are killing moves, meaning that they can only be executed in order to kill/capture a piece
      if possiblemoves[2][2] == 'blank':
        possiblemoves[2][2] = 'impossible'
      if possiblemoves[3][2] == 'blank':
        possiblemoves[3][2] = 'impossible'
      #The first 2 moves are non-killing moves, meaning that they cannot be executed in order to kill/capture a piece
      if possiblemoves[0][2] != 'blank':
        possiblemoves[0][2] = 'impossible'
        possiblemoves[1][2] = 'impossible'
      if possiblemoves[1][2] != 'blank':
        possiblemoves[1][2] = 'impossible'

    if len(possiblemoves)==3: #if not on first row
    #same as above, but with two killing moves
      if possiblemoves[1][2] == 'blank':
        possiblemoves[1][2] = 'impossible'
      if possiblemoves[2][2] == 'blank':
        possiblemoves[2][2] = 'impossible'
    
    return possiblemoves
  
  def rook(piece, board):
    row = piece[0]
    column = piece[1]
    possiblemoves = []    
    for i in range(8):
      possiblemoves.append([row+i, column])
    for i in range(8):
      possiblemoves.append([row-i, column])
    for i in range(8):
      possiblemoves.append([row, column+i])
    for i in range(8):
      possiblemoves.append([row, column-i])

    for possiblemove in possiblemoves:
      possiblemove.append(moves.verify(possiblemove, piece, board))
    
    for possiblemove in possiblemoves:
      
      if possiblemove[0]>row and possiblemove[1]==column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]>possiblemove[0] and possible[1]==possiblemove[1]:
            possible[2] = 'impossible'

      if possiblemove[0]<row and possiblemove[1]==column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]<possiblemove[0] and possible[1]==possiblemove[1]:
            possible[2] = 'impossible'    
      
      if possiblemove[0]==row and possiblemove[1]>column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]==possiblemove[0] and possible[1]>possiblemove[1]:
            possible[2] = 'impossible'

      if possiblemove[0]==row and possiblemove[1]<column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]==possiblemove[0] and possible[1]<possiblemove[1]:
            possible[2] = 'impossible'        
            
    return possiblemoves

  def bishop(piece, board):
    row = piece[0]
    column = piece[1]    
    possiblemoves = []
    for i in range(8):
      possiblemoves.append([row+i, column+i])
    for i in range(8):
      possiblemoves.append([row-i, column+i])
    for i in range(8):
      possiblemoves.append([row+i, column-i])
    for i in range(8):
      possiblemoves.append([row-i, column-i])


    for possiblemove in possiblemoves:
      possiblemove.append(moves.verify(possiblemove, piece, board))

    for possiblemove in possiblemoves:
      
      if possiblemove[0]>row and possiblemove[1]>column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]>possiblemove[0] and possible[1]>possiblemove[1]:
            possible[2] = 'impossible'

      if possiblemove[0]<row and possiblemove[1]<column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]<possiblemove[0] and possible[1]<possiblemove[1]:
            possible[2] = 'impossible'    
      
      if possiblemove[0]<row and possiblemove[1]>column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]<possiblemove[0] and possible[1]>possiblemove[1]:
            possible[2] = 'impossible'

      if possiblemove[0]>row and possiblemove[1]<column and possiblemove[2] != 'blank':
        for possible in possiblemoves:
          if possible[0]>possiblemove[0] and possible[1]<possiblemove[1]:
            possible[2] = 'impossible'
            
    return possiblemoves

  def knight(piece, board):
    row = piece[0]
    column = piece[1]
    possiblemoves = [[row+1, column+2], [row+1, column-2], [row-1, column-2], [row-1, column+2],[row+2, column-1], [row+2, column+1], [row-2, column+1], [row-2, column-1]] 
    for possiblemove in possiblemoves:
      possiblemove.append(moves.verify(possiblemove, piece, board))
    return possiblemoves

  def king(piece, board):
    row = piece[0]
    column = piece[1]
    possiblemoves = [[row+1, column], [row+1, column+1], [row, column+1], [row-1, column+1],[row-1, column], [row-1, column-1], [row, column-1], [row+1, column-1]] 
    for possiblemove in possiblemoves:
      possiblemove.append(moves.verify(possiblemove, piece, board))
    return possiblemoves      
  
  def queen(piece, board):
    possiblemoves = []
    possiblemovers = []
    possiblemovers = moves.rook(piece, board)
    for pos in possiblemovers:
        possiblemoves.append(pos)
    possiblemovers = moves.bishop(piece, board)
    for pos in possiblemovers:
        possiblemoves.append(pos)
    
    return possiblemoves

  def castle(identity, pieces, board):

    possiblecastles = []

    if identity == 'black':
      
      if pieces['Ib'][2] == False:
        if pieces['Rb1'][2] == False:
          if board[0,3] == 'blk' and board[0,2] == 'blk' and board[0,1] == 'blk':
            possiblecastles.append(['Ib', 0, 3 , 'Rb2', 0, 2])
        if pieces['Rb2'][2] == False:
          if board[0,5] == 'blk' and board[0,6] == 'blk':
            possiblecastles.append(['Ib', 0, 5 , 'Rb1', 0, 6])
    
    if identity == 'white':

      if pieces['Iw'][2] == False:
        if pieces['Rw1'][2] == False:
          if board[7,3] == 'blk' and board[7,2] == 'blk' and board[7,1] == 'blk':
            possiblecastles.append(['Ib', 7, 3 , 'Rw2', 7, 2])
        if pieces['Rb2'][2] == False:
          if board[7,5] == 'blk' and board[7,6] == 'blk':
            possiblecastles.append(['Iw', 7, 5 , 'Rw1', 7, 6])

    return possiblecastles

  def allmoves(identity, pieces, board):
    allmove = []
    keys = list(pieces.keys())
    if identity == 'white':
      for row in range(8):
        for column in range(8):
          if 'w' in board[row][column]:
            if 'P' in board[row][column]:
              allmove.append([board[row][column], moves.pawn([row,column], board)])
            if 'R' in board[row][column]:
              allmove.append([board[row][column], moves.rook([row,column], board)])
            if 'KN' in board[row][column]:
              allmove.append([board[row][column], moves.knight([row,column], board)])
            if 'B' in board[row][column]:
              allmove.append([board[row][column], moves.bishop([row,column], board)])
            if 'Q' in board[row][column]:
              allmove.append([board[row][column], moves.queen([row,column], board)])
            if 'I' in board[row][column]:
              allmove.append([board[row][column], moves.king([row,column], board)])

    if identity == 'black':
      for row in range(8):
        for column in range(8):
          if 'b' in board[row][column]:
            if 'P' in board[row][column]:
              allmove.append([board[row][column], moves.pawn([row,column], board)])
            if 'R' in board[row][column]:
              allmove.append([board[row][column], moves.rook([row,column], board)])
            if 'KN' in board[row][column]:
              allmove.append([board[row][column], moves.knight([row,column], board)])
            if 'B' in board[row][column]:
              allmove.append([board[row][column], moves.bishop([row,column], board)])
            if 'Q' in board[row][column]:
              allmove.append([board[row][column], moves.queen([row,column], board)])
            if 'I' in board[row][column]:
              allmove.append([board[row][column], moves.king([row,column], board)])

    allmoves = []

    for i in allmove:
      allmoves.append([i[0]])
      for c in i[1]:
        if c[2] != 'impossible':
          allmoves[len(allmoves)-1].append(c)

    allmoves.append(['Castle'])
    for asd in moves.castle(identity, pieces, board):
      allmoves[len(allmoves)-1].append(asd)

    return allmoves


  def tester(identity, move, pieces, board):

    chess_board = copy.deepcopy(board)
    check = False
    #Creates Temporary Vars for Castle (Weird System)
    if move[0] == 'Castle':

      name = move[1][0]
      name_row = move[1][1]
      name_column = move[1][2]
      move_to = move[1][3]
      move_row = move[1][4]
      move_column = move[1][5]

    #Creates Temporary Vars for any other type
    else:
      name = move[0]
      name_row = 0
      name_column = 0

      for i in range(len(chess_board)):
        for c in range(len(chess_board[i])):
          if chess_board[i][c] == name:
            name_row = i
            name_column = c


      move_row = move[1][0]
      move_column = move[1][1]
      move_to = move[1][2]
      captured = 'None'
      

    #Makes a fake board (chess_board) with the move implemented
    #If move is blank, just move it to the blank spot
    if move_to == 'blank':
      chess_board[move_row][move_column] = copy.deepcopy(name)
      chess_board[name_row][name_column] = 'blk'

    #If the move is capturing something, make 'captured' whatever its capturing
    else:
      chess_board[move_row][move_column] = copy.deepcopy(name)
      chess_board[name_row][name_column] = 'blk'
      captured = move_to

    #If it's black, using the fake board with the move, find the moves of the other side
    #and check whether the king is in any of that side's moves
    if identity == 'black':
      all = moves.allmoves('white', pieces, chess_board)
      for move in all:
        if move[0]!='castle':
          for d in move[1:]:
            chessboard = copy.deepcopy(board)
            if d[2] == 'Ib':
              return 'impossible', captured


    #Same for white
    if identity == 'white':
      all = moves.allmoves('black', pieces, chess_board)
      for newmove in all:
        if newmove[0]!='castle':
          for d in newmove[1:]:
            chessboard = copy.deepcopy(board)
            if d[2] == 'Iw':
              return 'impossible', captured

    return 'possible', captured
    


  def allmovesfinalimplement(identity, pieces, board):
    all = moves.allmoves(identity, pieces, board)
    newall = []
    for move in all:
      for c in move[1:]:
        thr, fou = moves.tester(identity, [move[0], c], pieces, board)
        c.append(thr)
        c.append(fou)
        c.insert(0, move[0])
        if c[len(c)-2] == 'possible':
          newall.append(c)
        
    

    return newall

  def checkbool(sign, piece, board):
    check = False
    if sign=="black": 
      temp = moves.allmovesfinalimplement("white", pieces, board)
      for i in range(len(moves.allmovesfinalimplement("white", pieces, board))):
        for j in range(len(temp[i])):
          if type(temp[i][j]) == str:
            continue
          elif type(temp[i][j]) == list:
            if temp[i][j][2] == "Ib":
              check = True
    elif sign == "white":
      temp = moves.allmovesfinalimplement("black", pieces, board)
      for i in range(len(moves.allmovesfinalimplement("black", pieces, board))):
            if temp[i][3] == "Iw":
              check = True
    return check

  def CheckStaleMate(sign, pieces, board):
   if len(moves.allmovesfinalimplement(sign, pieces, board)) ==0:
     check = moves.checkbool(sign, pieces, board)
     if check == True:
       return f"{str(sign)} has lost."
     elif check == False:
       return 'Draw'
   else:
     return False

  def movement(move, identity, pieces, board):
    name = move[0]
    
    if name!='Castle':
      name_row = 0
      name_column = 0
      move_row = move[1]
      move_column = move[2]
      captured = move[5]

      for i in range(len(board)):
        for c in range(len(board[i])):
          if board[i][c] == name:
            name_row = i
            name_column = c

      if 'I' in name or 'R' in name:
        pieces[name][2] = 'Moved'
      
      if captured!='None':
        pieces[captured][1] = 'Captured'

      board[name_row][name_column] = 'blk'
      board[move_row][move_column] = name
      return move, pieces, board, captured
    
    else:
      name = move[1]
      name_row = move[2]
      name_column = move[3]
      name_prev_row = 0
      name_prev_column = 0

      for i in range(len(board)):
        for c in range(len(board[i])):
          if board[i][c] == name:
            name_prev_row = i
            name_prev_column = c

      move_name = move[4]
      move_row = move[5]
      move_column = move[6]
      move_prev_row = 0
      move_prev_column = 0

      for i in range(len(board)):
        for c in range(len(board[i])):
          if board[i][c] == move_name:
            move_prev_row = i
            move_prev_column = c

      captured = move[8]

      pieces[name][2] = 'Moved'
      pieces[move_name][2] = 'Moved'

      print(board)
      board[name_row][name_column] = move_name
      board[move_row][move_column] = name
      board[move_prev_row][move_prev_column] = 'blk'
      board[name_prev_row][name_prev_column] = 'blk'
      return move, pieces, board, captured

pieces, board = frameboard.createboard()
team = 'white'
if moves.CheckStaleMate(team, pieces, board) == False:
  print(moves.allmovesfinalimplement(team, pieces, board))
else:
  print(moves.CheckStaleMate(team, pieces, board))

[['Pw1', 5, 0, 'blank', 'possible', 'None'], ['Pw1', 4, 0, 'blank', 'possible', 'None'], ['Pw2', 5, 1, 'blank', 'possible', 'None'], ['Pw2', 4, 1, 'blank', 'possible', 'None'], ['Pw3', 5, 2, 'blank', 'possible', 'None'], ['Pw3', 4, 2, 'blank', 'possible', 'None'], ['Pw4', 5, 3, 'blank', 'possible', 'None'], ['Pw4', 4, 3, 'blank', 'possible', 'None'], ['Pw5', 5, 4, 'blank', 'possible', 'None'], ['Pw5', 4, 4, 'blank', 'possible', 'None'], ['Pw6', 5, 5, 'blank', 'possible', 'None'], ['Pw6', 4, 5, 'blank', 'possible', 'None'], ['Pw7', 5, 6, 'blank', 'possible', 'None'], ['Pw7', 4, 6, 'blank', 'possible', 'None'], ['Pw8', 5, 7, 'blank', 'possible', 'None'], ['Pw8', 4, 7, 'blank', 'possible', 'None'], ['KNw1', 5, 2, 'blank', 'possible', 'None'], ['KNw1', 5, 0, 'blank', 'possible', 'None'], ['KNw2', 5, 7, 'blank', 'possible', 'None'], ['KNw2', 5, 5, 'blank', 'possible', 'None']]
